In [1]:
import numpy as np
import matplotlib.pyplot as plt
import support_code

In [32]:
data = np.load('./ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

# 1

## a)

In [34]:
print(f'The mean of the training positions in y_train is {np.mean(y_train):.4f} ± {np.var(y_train)/np.sqrt(len(y_train)):.4f}.')
print(f'The mean of the 5,785 positions in the y_val is {np.mean(y_val):.4f} ± {np.var(y_val)/np.sqrt(len(y_val)):.4f}.')
print(f'The mean of the first 5,785 entries in the y_train is {np.mean(y_train[0:5785,]):.4f} ± {np.var(y_train[0:5785,])/np.sqrt(5785):.4f}.')

The mean of the training positions in y_train is -0.0000 ± 0.0050.
The mean of the 5,785 positions in the y_val is -0.2160 ± 0.0127.
The mean of the first 5,785 entries in the y_train is -0.4425 ± 0.0108.


## b)

In [33]:
index = np.var(X_train, axis=0) != 0

X_train = X_train[:, index]
X_val = X_val[:, index]
X_test = X_test[:, index]

# 2

## a)

In [ ]:
def fit_linreg(X, yy, alpha):
    n, d = X.shape[0], X.shape[1]
    y_tilde = np.concatenate((yy, np.zeros(d)))
    Phi = np.concatenate((X, np.ones((n, 1))), axis=1)
    diag = np.diag(np.append(np.sqrt(alpha) * np.ones((d)), [0]))
    Phi_tilde = np.concatenate((Phi, diag))
    y_tilde = np.concatenate((yy, np.zeros((d+1,1))))
    w_tilde = np.linalg.lstsq(Phi_tilde, y_tilde)
    ww, bb = w_tilde[0:d], w_tilde[d]
    return ww, bb

In [ ]:
# fit X_train and y_train with alpha = 30

In [ ]:
# Try the gradient decent method
# support_code.fit_linreg_gradopt(X, yy, 30)

In [ ]:
# Report RMSE = sqrt(||y - f(x;w,b)||^2 / N)